# Lisbon Neighborhoods

## First Step - Scraping

In [1]:
# libraries to import
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# http://www.geonames.org/postalcode-search.html?q=1150&country=PT
# https://www.codigo-postal.pt/lisboa/
# https://opendata.arcgis.com/datasets/e0ebb7f5038e4114979f73cbf66321ef_1.geojson
# https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?outFields=*&where=1%3D1
#https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=
#https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=true&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token=
PageToScrape = requests.get('https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=').text
PageToScrape = BeautifulSoup(PageToScrape, 'xml')
PageToScrape = PageToScrape.find_all('table',{'class':'ftrTable'})
#PageToScrape = PageToScrape.find_all('tr')
#PageToScrape


In [3]:
table = []
for row in PageToScrape:
    table.append([t.text.strip() for t in row.find_all('td')])
#df = pd.DataFrame(table, columns=['OBJECTID', 'NOME'])
df = pd.DataFrame(table)
df = df.drop([0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], axis=1)
df.head()


,5,21
0,Santo António,Point:\nX: -1018440.6546671605\nY: 4681834.853...
1,Parque das Nações,Point:\nX: -1012503.9786498934\nY: 4689466.068...
2,Marvila,Point:\nX: -1014226.7638454303\nY: 4685629.513...
3,Ajuda,Point:\nX: -1023989.21146323\nY: 4680527.21680...
4,Areeiro,Point:\nX: -1016739.3330696875\nY: 4684693.942...


In [4]:
df[21] = df[21].str.replace("Point:\n", "")
# new data frame with split value columns 
new = df[21].str.split("\n", n = 1, expand = True) 
# making separate first name column from new data frame 
df["X"]= new[0] 
# making separate last name column from new data frame 
df["Y"]= new[1]
# Dropping old Name columns 
df.drop(columns =[21], inplace = True) 
df["X"] = df["X"].str.replace("X: ", "")
df["Y"] = df["Y"].str.replace("Y: ", "")
df.rename(columns={5: 'Freguesia'}, inplace=True)
df.head()

,Freguesia,X,Y
0,Santo António,-1018440.6546671605,4681834.85372911
1,Parque das Nações,-1012503.9786498934,4689466.0681953141
2,Marvila,-1014226.7638454303,4685629.5130308038
3,Ajuda,-1023989.21146323,4680527.2168011209
4,Areeiro,-1016739.3330696875,4684693.9425387029


In [5]:
!pip install pyproj
from pyproj import Proj, transform

def function(row):
    inProj = Proj(init='epsg:3857')
    outProj = Proj(init='epsg:4326')
    z2 = transform(inProj,outProj,row.X, row.Y)
    return z2

df['GPS'] = df.apply(function, axis=1)
df.head()

     |████████████████████████████████| 10.9MB 6.4MB/s eta 0:00:01


/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.

,Freguesia,X,Y,GPS
0,Santo António,-1018440.6546671605,4681834.85372911,"(-9.148808060562017, 38.72134487346995)"
1,Parque das Nações,-1012503.9786498934,4689466.0681953141,"(-9.095477992530249, 38.774809235541774)"
2,Marvila,-1014226.7638454303,4685629.5130308038,"(-9.110954035254304, 38.74793529763537)"
3,Ajuda,-1023989.21146323,4680527.2168011209,"(-9.19865159430916, 38.712179538844204)"
4,Areeiro,-1016739.3330696875,4684693.9425387029,"(-9.13352482861989, 38.741380367314136)"


In [6]:
df1 = df
df.head()

,Freguesia,X,Y,GPS
0,Santo António,-1018440.6546671605,4681834.85372911,"(-9.148808060562017, 38.72134487346995)"
1,Parque das Nações,-1012503.9786498934,4689466.0681953141,"(-9.095477992530249, 38.774809235541774)"
2,Marvila,-1014226.7638454303,4685629.5130308038,"(-9.110954035254304, 38.74793529763537)"
3,Ajuda,-1023989.21146323,4680527.2168011209,"(-9.19865159430916, 38.712179538844204)"
4,Areeiro,-1016739.3330696875,4684693.9425387029,"(-9.13352482861989, 38.741380367314136)"


In [7]:
df1.head()
df1[["GPS"]] = df1[["GPS"]].astype("str")
df1['GPS'] = df1['GPS'].str.replace("(", "")
df1['GPS'] = df1['GPS'].str.replace(")", "")
#df1.head()
#df1.dtypes
new = df1['GPS'].str.split(",", n = 1, expand = True) 
df1.head()
# making separate first name column from new data frame 
df1["Longitude"]= new[0] 
df1.head()
#df1.head()
# making separate last name column from new data frame 
df1["Latitude"]= new[1]
# Dropping old Name columns 
df1.drop(columns =['GPS'], inplace = True) 
df1.head()

,Freguesia,X,Y,Longitude,Latitude
0,Santo António,-1018440.6546671605,4681834.85372911,-9.148808060562017,38.72134487346995
1,Parque das Nações,-1012503.9786498934,4689466.0681953141,-9.095477992530249,38.774809235541774
2,Marvila,-1014226.7638454303,4685629.5130308038,-9.110954035254304,38.74793529763537
3,Ajuda,-1023989.21146323,4680527.2168011209,-9.19865159430916,38.712179538844204
4,Areeiro,-1016739.3330696875,4684693.9425387029,-9.13352482861989,38.741380367314136


In [8]:
df1 = df1.drop(['X', 'Y'], axis=1)
df1.head()

,Freguesia,Longitude,Latitude
0,Santo António,-9.148808060562017,38.72134487346995
1,Parque das Nações,-9.095477992530249,38.774809235541774
2,Marvila,-9.110954035254304,38.74793529763537
3,Ajuda,-9.19865159430916,38.712179538844204
4,Areeiro,-9.13352482861989,38.741380367314136


In [9]:
CLIENT_ID = 'KSAQZZJC23YINY1EVGBP2O1FI1IWWNEAV4WTZSWW2LIWEW5M' # your Foursquare ID
CLIENT_SECRET = 'VEIFWQ4COHJI31OWZSDTTG0FWCA3NS34UI45P2PXZ5Y4VJ5K' # your Foursquare Secret
VERSION = '20180604'

In [11]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
lisbon_venues = getNearbyVenues(names=df1['Freguesia'],
                                   latitudes=df1['Latitude'],
                                   longitudes=df1['Longitude']
                                  )


Santo António
Parque das Nações
Marvila
Ajuda
Areeiro
Santa Maria Maior
Alvalade
Belém
Estrela
Arroios
Santa Clara
Avenidas Novas
Carnide
São Domingos de Benfica
Beato
Campolide
Alcântara
Campo de Ourique
São Vicente
Olivais
Misericórdia
Lumiar
Penha de França
Benfica


In [14]:
print(lisbon_venues.shape)
lisbon_venues.head()
condition = lisbon_venues['Neighborhood'] == 'Alcântara'
filtro = lisbon_venues[condition]
print(filtro.shape)
filtro.head(10)

(1702, 7)
(62, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1156,Alcântara,38.709600048975645,-9.183388632388784,A Tapadinha,38.706576,-9.178730,Eastern European Restaurant
1157,Alcântara,38.709600048975645,-9.183388632388784,Pestana Palace Hotel & National Monument,38.703687,-9.186756,Hotel
1158,Alcântara,38.709600048975645,-9.183388632388784,Jardim do Alto de Santo Amaro,38.704570,-9.184478,Garden
1159,Alcântara,38.709600048975645,-9.183388632388784,LX Factory,38.703091,-9.178833,General Entertainment
1160,Alcântara,38.709600048975645,-9.183388632388784,Landeau Chocolate,38.702888,-9.178719,Dessert Shop
1161,Alcântara,38.709600048975645,-9.183388632388784,Solar dos Nunes,38.705364,-9.180578,Mediterranean Restaurant
1162,Alcântara,38.709600048975645,-9.183388632388784,Ler Devagar,38.702752,-9.178520,Bookstore
1163,Alcântara,38.709600048975645,-9.183388632388784,Lidl,38.705046,-9.179043,Supermarket
1164,Alcântara,38.709600048975645,-9.183388632388784,O Mercado,38.705005,-9.178732,Mediterranean Restaurant
1165,Alcântara,38.709600048975645,-9.183388632388784,O Bolo da Marta,38.702696,-9.178399,Bakery


In [15]:
print(lisbon_venues.shape)
lisbon_venues.groupby('Neighborhood').count()

(1702, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ajuda,21,21,21,21,21,21
Alcântara,62,62,62,62,62,62
Alvalade,100,100,100,100,100,100
Areeiro,100,100,100,100,100,100
Arroios,100,100,100,100,100,100
Avenidas Novas,100,100,100,100,100,100
Beato,20,20,20,20,20,20
Belém,82,82,82,82,82,82
Benfica,10,10,10,10,10,10


In [16]:
df_count = lisbon_venues.groupby('Neighborhood').count()
df_count.drop(columns =['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], inplace = True) 
df_count.rename(columns={'Venue': 'Count'}, inplace=True) 

condition = df_count['Count'] > 20
df_count = df_count[condition]

df_count.head()

lisbon_venues = lisbon_venues.merge(df_count, left_on='Neighborhood', right_on='Neighborhood')
lisbon_venues.drop(columns =['Count'], inplace = True) 


print(lisbon_venues.shape)
lisbon_venues.groupby('Neighborhood').count()


(1643, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ajuda,21,21,21,21,21,21
Alcântara,62,62,62,62,62,62
Alvalade,100,100,100,100,100,100
Areeiro,100,100,100,100,100,100
Arroios,100,100,100,100,100,100
Avenidas Novas,100,100,100,100,100,100
Belém,82,82,82,82,82,82
Campo de Ourique,100,100,100,100,100,100
Campolide,64,64,64,64,64,64


In [17]:
# one hot encoding
lisbon_onehot = pd.get_dummies(lisbon_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
lisbon_onehot['Neighborhood'] = lisbon_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [lisbon_onehot.columns[-1]] + list(lisbon_onehot.columns[:-1])
lisbon_onehot = lisbon_onehot[fixed_columns]
lisbon_onehot.head() 


,Zoo,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Tibetan Restaurant,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Waterfront,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
lisbon_grouped = lisbon_onehot.groupby('Neighborhood').mean().reset_index()
lisbon_grouped.head()


,Neighborhood,Zoo,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tibetan Restaurant,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Waterfront,Wine Bar,Wine Shop,Wings Joint
0,Ajuda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00
1,Alcântara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00
2,Alvalade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00
3,Areeiro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.01
4,Arroios,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.00


In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        # create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lisbon_grouped['Neighborhood']

for ind in np.arange(lisbon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lisbon_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ajuda,Restaurant,Park,Pharmacy,Supermarket,Historic Site,Gas Station,Football Stadium,Coffee Shop,Gym,Café
1,Alcântara,Restaurant,Portuguese Restaurant,Café,Bakery,Coffee Shop,Dessert Shop,BBQ Joint,Indian Restaurant,Hotel,Mediterranean Restaurant
2,Alvalade,Portuguese Restaurant,Restaurant,Plaza,Café,Coffee Shop,Bar,Bakery,Chinese Restaurant,Burger Joint,Italian Restaurant
3,Areeiro,Portuguese Restaurant,Coffee Shop,Electronics Store,Plaza,Ice Cream Shop,Hotel,Bar,BBQ Joint,Italian Restaurant,Bakery
4,Arroios,Portuguese Restaurant,Hotel,Café,Indian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Restaurant,Hotel Bar


In [20]:
import os
from sklearn.cluster import KMeans


In [38]:
# set number of clusters
kclusters = 5
lisbon_grouped_clustering = lisbon_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lisbon_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([4, 2, 2, 2, 0, 2, 2, 2, 0, 3], dtype=int32)

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Ajuda,Restaurant,Park,Pharmacy,Supermarket,Historic Site,Gas Station,Football Stadium,Coffee Shop,Gym,Café
1,2,Alcântara,Restaurant,Portuguese Restaurant,Café,Bakery,Coffee Shop,Dessert Shop,BBQ Joint,Indian Restaurant,Hotel,Mediterranean Restaurant
2,2,Alvalade,Portuguese Restaurant,Restaurant,Plaza,Café,Coffee Shop,Bar,Bakery,Chinese Restaurant,Burger Joint,Italian Restaurant
3,2,Areeiro,Portuguese Restaurant,Coffee Shop,Electronics Store,Plaza,Ice Cream Shop,Hotel,Bar,BBQ Joint,Italian Restaurant,Bakery
4,0,Arroios,Portuguese Restaurant,Hotel,Café,Indian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Restaurant,Hotel Bar
5,2,Avenidas Novas,Restaurant,Portuguese Restaurant,Vegetarian / Vegan Restaurant,Hotel,Coffee Shop,Steakhouse,Italian Restaurant,Gym / Fitness Center,Café,Bakery
6,2,Belém,Portuguese Restaurant,Monument / Landmark,Restaurant,Garden,Bakery,Café,Ice Cream Shop,Burger Joint,Mediterranean Restaurant,Food Truck
7,2,Campo de Ourique,Portuguese Restaurant,Restaurant,Seafood Restaurant,Bakery,Coffee Shop,Café,Bar,Hotel,Bed & Breakfast,Scenic Lookout
8,0,Campolide,Hotel,Portuguese Restaurant,Restaurant,Bakery,Park,Japanese Restaurant,Sushi Restaurant,Café,Coffee Shop,Theater
9,3,Carnide,Portuguese Restaurant,Café,Restaurant,Bakery,Theater,Pizza Place,Supermarket,Italian Restaurant,Soccer Field,Bar


In [40]:
lisbon_merged = df1
lisbon_merged.rename(columns={'Freguesia':'Neighborhood'},inplace=True)
# merge lisbon_grouped with lisbon_data to add latitude/longitude for each neighborhood
lisbon_merged = lisbon_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
lisbon_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
lisbon_merged["Cluster Labels"]=lisbon_merged["Cluster Labels"].astype("int")
print(lisbon_merged.head(5))


        Neighborhood           Longitude             Latitude  Cluster Labels  \
0      Santo António  -9.148808060562017    38.72134487346995               0   
1  Parque das Nações  -9.095477992530249   38.774809235541774               2   
3              Ajuda   -9.19865159430916   38.712179538844204               4   
4            Areeiro   -9.13352482861989   38.741380367314136               2   
5  Santa Maria Maior  -9.132403842442532   38.708311728166244               3   

   1st Most Common Venue  2nd Most Common Venue 3rd Most Common Venue  \
0                  Hotel  Portuguese Restaurant                  Café   
1  Portuguese Restaurant                   Café          Burger Joint   
3             Restaurant                   Park              Pharmacy   
4  Portuguese Restaurant            Coffee Shop     Electronics Store   
5  Portuguese Restaurant                  Hotel                 Plaza   

  4th Most Common Venue 5th Most Common Venue          6th Most Common Ven

In [25]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [26]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors


In [41]:
address = 'Seixal, PT'
geolocator = Nominatim(user_agent="lisbon")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seixal are {}, {}.'.format(latitude, longitude))

address = 'Lisbon, PT'
geolocator = Nominatim(user_agent="lisbon")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lisbon are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Seixal are 38.6422246, -9.1072662.
The geograpical coordinate of Lisbon are 38.7077507, -9.1365919.


In [42]:
lisbon_merged[["Longitude"]] = lisbon_merged[["Longitude"]].astype("float")
lisbon_merged[["Latitude"]] = lisbon_merged[["Latitude"]].astype("float")
lisbon_merged.head(25)

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Santo António,-9.148808,38.721345,0,Hotel,Portuguese Restaurant,Café,Gym,Breakfast Spot,Vegetarian / Vegan Restaurant,Coffee Shop,Garden,Italian Restaurant,Bar
1,Parque das Nações,-9.095478,38.774809,2,Portuguese Restaurant,Café,Burger Joint,Coffee Shop,Electronics Store,Restaurant,Gym,Seafood Restaurant,Hotel,Ice Cream Shop
3,Ajuda,-9.198652,38.712180,4,Restaurant,Park,Pharmacy,Supermarket,Historic Site,Gas Station,Football Stadium,Coffee Shop,Gym,Café
4,Areeiro,-9.133525,38.741380,2,Portuguese Restaurant,Coffee Shop,Electronics Store,Plaza,Ice Cream Shop,Hotel,Bar,BBQ Joint,Italian Restaurant,Bakery
5,Santa Maria Maior,-9.132404,38.708312,3,Portuguese Restaurant,Hotel,Plaza,Ice Cream Shop,Café,Scenic Lookout,Bar,Restaurant,Wine Bar,Hostel
6,Alvalade,-9.146497,38.753889,2,Portuguese Restaurant,Restaurant,Plaza,Café,Coffee Shop,Bar,Bakery,Chinese Restaurant,Burger Joint,Italian Restaurant
7,Belém,-9.213136,38.696209,2,Portuguese Restaurant,Monument / Landmark,Restaurant,Garden,Bakery,Café,Ice Cream Shop,Burger Joint,Mediterranean Restaurant,Food Truck
8,Estrela,-9.163458,38.703768,2,Portuguese Restaurant,Nightclub,Restaurant,Hotel,Coffee Shop,Hostel,Tapas Restaurant,Café,Italian Restaurant,Plaza
9,Arroios,-9.137940,38.727658,0,Portuguese Restaurant,Hotel,Café,Indian Restaurant,Bakery,Vegetarian / Vegan Restaurant,Gym / Fitness Center,Gym,Restaurant,Hotel Bar
11,Avenidas Novas,-9.152328,38.737465,2,Restaurant,Portuguese Restaurant,Vegetarian / Vegan Restaurant,Hotel,Coffee Shop,Steakhouse,Italian Restaurant,Gym / Fitness Center,Café,Bakery


In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lisbon_merged['Latitude'], lisbon_merged['Longitude'], lisbon_merged['Neighborhood'], lisbon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(label)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


##  Where would you recommend that they open a restaurant?

I would recommend a neighborhood with a lot of hotels/ hostels or with important monuments or landmark. Therefore I would choose Santo António. An important factor is the rental price, which was not considered yet.


## Where would you recommend that they setup the office for your own business?

I think I do not have the necessary information because Foursquare is to much centered in restaurants, bars and things like that. That infomration is not enough for the decision for a business. What kind of business?